# Segmenting and Clustering Neighborhoods in Toronto: PART THREE

## By Katarzyna Cybulska

This is notebook three of three containing steps in segmentation and clustering of neighborhoods in Toronto. In this workbook we will find segment and cluster Toronto neighborhoods based on venues you can find in them. Enjoy!

# Import additional libraries

In [7]:
#import packages

import numpy as np # library to handle data in a vectorized manner

import pandas as pd # library for data analsysis
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors


# import k-means from clustering stage
from sklearn.cluster import KMeans

!conda install -c conda-forge folium=0.5.0 --yes
import folium # map rendering library

!conda install -c conda-forge geopy --yes # another geolocator, which works fine on plain text
from geopy.geocoders import Nominatim 



print('Libraries imported.')


Solving environment: done

## Package Plan ##

  environment location: /home/jupyterlab/conda

  added / updated specs: 
    - folium=0.5.0


The following packages will be downloaded:

    package                    |            build
    ---------------------------|-----------------
    altair-2.2.2               |           py36_1         461 KB  conda-forge
    vincent-0.4.4              |             py_1          28 KB  conda-forge
    certifi-2018.8.24          |        py36_1001         139 KB  conda-forge
    branca-0.3.0               |             py_0          24 KB  conda-forge
    folium-0.5.0               |             py_0          45 KB  conda-forge
    ------------------------------------------------------------
                                           Total:         697 KB

The following NEW packages will be INSTALLED:

    altair:  2.2.2-py36_1     conda-forge
    branca:  0.3.0-py_0       conda-forge
    folium:  0.5.0-py_0       conda-forge
    vincent: 0.4.4-p

# Visualize set of data pulled out from Wikipedia. 

In [66]:
#save toronto_neighborhoods under new variable (I will use only Boroughs which contain "Toronto" in their name)
hoods_df = toronto_neighborhoods[toronto_neighborhoods['Borough'].str.contains("Toronto")]


#initialize geolocator object
geolocator = Nominatim(user_agent = "Kasia")

# Find coordinates of Toronto
address = 'Toronto'
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude

# Initialize map of Torronro
map_toronto = folium.Map(location=[latitude, longitude], zoom_start=12)

# add markers to map
for lat, lng, borough, neighborhood in zip(hoods_df['Latitude'], hoods_df['Longitude'], hoods_df['Borough'],hoods_df['Neighborhood']):
    label = "{}, ({})".format(borough,neighborhood)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_toronto)  
    
map_toronto

 # Maps are not viewed correctly in github. Please follow the link to see the map:
https://github.com/warstwowa/Coursera_Capstone/blob/master/map%201.PNG

# Explore neighborhoods with Foursquare.
Put in your Foursquare credentials.

In [67]:
# Start exploring them with Foursquare.

CLIENT_ID = 'client_id' # your Foursquare ID
CLIENT_SECRET = 'secrted'
VERSION = '20180923' # Foursquare API version
LIMIT = 100
radius = 500

print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Your credentails:
CLIENT_ID: client_id
CLIENT_SECRET:secrted


# Get nearby venues in all neighborhoods.
First define getNearbyVenues function which will return nice looking data frame and then run it on hoods_df data frame.

In [10]:
# Functions defined in previous lab will be usefull to get all nearby venues.

def getNearbyVenues(names, latitudes, longitudes, radius=500):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

# Save nearby venues in a data frame.
Call this data frame "toronto_venues"

In [11]:
toronto_venues = getNearbyVenues(names=hoods_df['Neighborhood'],
                                   latitudes=hoods_df['Latitude'],
                                   longitudes=hoods_df['Longitude'])

Harbourfront
Ryerson
St. James Town
The Beaches
Berczy Park
Central Bay Street
Christie
Adelaide, King
Dovercourt Village
Harbourfront East, Toronto Islands
Little Portugal
The Danforth West
Design Exchange
Brockton, Exhibition Place
The Beaches West
Commerce Court
Studio District
Lawrence Park
Roselawn
Davisville North
Forest Hill North
High Park
North Toronto West
The Annex, North Midtown
Parkdale
Davisville
Harbord
Runnymede
Moore Park
Chinatown, Grange Park
Deer Park, Forest Hill SE, Rathnelly, South Hill
CN Tower, Bathurst Quay, Island airport, Harbourfront West, King and Spadina, Railway Lands
Rosedale
Stn A PO Boxes 25 The Esplanade
Cabbagetown
First Canadian Place
Church and Wellesley
Business reply mail Processing Centre969 Eastern


# All nearby venues are stored in toronto_venues data frame.
Explore and reshape this data frame so that it's suitable for clustering.

In [12]:
#Let's see how many results from Forsqure did we get.
print(toronto_venues.shape)
toronto_venues.head()

(1698, 7)


,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,Harbourfront,43.65426,-79.360636,Roselle Desserts,43.653447,-79.362017,Bakery
1,Harbourfront,43.65426,-79.360636,Tandem Coffee,43.653559,-79.361809,Coffee Shop
2,Harbourfront,43.65426,-79.360636,Body Blitz Spa East,43.654735,-79.359874,Spa
3,Harbourfront,43.65426,-79.360636,Morning Glory Cafe,43.653947,-79.361149,Breakfast Spot
4,Harbourfront,43.65426,-79.360636,Cooper Koo YMCA,43.653191,-79.357947,Gym / Fitness Center


# Check how many results are there in each neighborhood.

In [15]:
# Using .groupby() method to get number of results in each neighbourhood.

print("There are {} neighborhoods.".format(toronto_venues.groupby('Neighborhood').count().shape[0]))
toronto_venues.groupby('Neighborhood').count()


There are 38 neighborhoods.


,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
Neighborhood,,,,,,
"Adelaide, King",100,100,100,100,100,100
Berczy Park,55,55,55,55,55,55
"Brockton, Exhibition Place",22,22,22,22,22,22
Business reply mail Processing Centre969 Eastern,16,16,16,16,16,16
"CN Tower, Bathurst Quay, Island airport, Harbourfront West, King and Spadina, Railway Lands",14,14,14,14,14,14
Cabbagetown,47,47,47,47,47,47
Central Bay Street,83,83,83,83,83,83
"Chinatown, Grange Park",100,100,100,100,100,100
Christie,15,15,15,15,15,15


# Reshape data frame with one hot encoding.

In [16]:
# one hot encoding
toronto_onehot = pd.get_dummies(toronto_venues[['Venue Category']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
toronto_onehot['Neighborhood'] = toronto_venues['Neighborhood'] 

# move neighborhood column to the first column
fixed_columns = [toronto_onehot.columns[-1]] + list(toronto_onehot.columns[:-1])
toronto_onehot = toronto_onehot[fixed_columns]

toronto_onehot.head()

,Yoga Studio,Accessories Store,Adult Boutique,Afghan Restaurant,Airport,Airport Food Court,Airport Gate,Airport Lounge,Airport Service,Airport Terminal,American Restaurant,Antique Shop,Aquarium,Arepa Restaurant,Art Gallery,Art Museum,Arts & Crafts Store,Asian Restaurant,Auto Workshop,BBQ Joint,Baby Store,Bagel Shop,Bakery,Bank,Bar,Baseball Stadium,Basketball Stadium,Beach,Beer Bar,Beer Store,Belgian Restaurant,Bistro,Boat or Ferry,Bookstore,Boutique,Brazilian Restaurant,Breakfast Spot,Brewery,Bubble Tea Shop,Building,Burger Joint,Burrito Place,Bus Line,Butcher,Café,Cajun / Creole Restaurant,Caribbean Restaurant,Cheese Shop,Chinese Restaurant,Chocolate Shop,Church,Climbing Gym,Clothing Store,Cocktail Bar,Coffee Shop,College Arts Building,College Gym,College Rec Center,Comfort Food Restaurant,Comic Shop,Concert Hall,Convenience Store,Cosmetics Shop,Coworking Space,Creperie,Cuban Restaurant,Cupcake Shop,Dance Studio,Deli / Bodega,Department Store,Dessert Shop,Dim Sum Restaurant,Diner,Discount Store,Dog Run,Doner Restaurant,Donut Shop,Dumpling Restaurant,Eastern European Restaurant,Electronics Store,Ethiopian Restaurant,Event Space,Falafel Restaurant,Farmers Market,Fast Food Restaurant,Filipino Restaurant,Fish & Chips Shop,Fish Market,Flea Market,Flower Shop,Food,Food & Drink Shop,Food Court,Food Truck,Fountain,French Restaurant,Fried Chicken Joint,Fruit & Vegetable Store,Furniture / Home Store,Gaming Cafe,Garden,Garden Center,Gas Station,Gastropub,Gay Bar,General Entertainment,General Travel,German Restaurant,Gift Shop,Gluten-free Restaurant,Gourmet Shop,Greek Restaurant,Grocery Store,Gym,Gym / Fitness Center,Harbor / Marina,Hawaiian Restaurant,Health & Beauty Service,Health Food Store,Historic Site,History Museum,Hobby Shop,Home Service,Hookah Bar,Hostel,Hotel,Hotel Bar,Hotpot Restaurant,Ice Cream Shop,Indian Restaurant,Indie Movie Theater,Intersection,Irish Pub,Italian Restaurant,Japanese Restaurant,Jazz Club,Jewelry Store,Jewish Restaurant,Juice Bar,Korean Restaurant,Lake,Latin American Restaurant,Light Rail Station,Lingerie Store,Liquor Store,Lounge,Mac & Cheese Joint,Malay Restaurant,Market,Martial Arts Dojo,Mediterranean Restaurant,Men's Store,Metro Station,Mexican Restaurant,Middle Eastern Restaurant,Miscellaneous Shop,Modern European Restaurant,Molecular Gastronomy Restaurant,Monument / Landmark,Movie Theater,Museum,Music Venue,Neighborhood,New American Restaurant,Nightclub,Noodle House,Office,Opera House,Optical Shop,Organic Grocery,Park,Performing Arts Venue,Persian Restaurant,Pet Store,Pharmacy,Piano Bar,Pizza Place,Plane,Playground,Plaza,Portuguese Restaurant,Poutine Place,Pub,Ramen Restaurant,Record Shop,Recording Studio,Rental Car Location,Restaurant,Sake Bar,Salad Place,Salon / Barbershop,Sandwich Place,Scenic Lookout,Sculpture Garden,Seafood Restaurant,Shoe Store,Shopping Mall,Skate Park,Skating Rink,Smoke Shop,Smoothie Shop,Snack Place,Soup Place,Southern / Soul Food Restaurant,Spa,Speakeasy,Sporting Goods Shop,Sports Bar,Stadium,Stationery Store,Steakhouse,Strip Club,Supermarket,Sushi Restaurant,Swim School,Taco Place,Tailor Shop,Taiwanese Restaurant,Tanning Salon,Tapas Restaurant,Tea Room,Tennis Court,Thai Restaurant,Theater,Thrift / Vintage Store,Toy / Game Store,Trail,Train Station,Vegetarian / Vegan Restaurant,Video Game Store,Vietnamese Restaurant,Wine Bar,Wings Joint,Women's Store
0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,Harbourfront,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,

# Group by neighborhoods.

In [17]:
toronto_grouped = toronto_onehot.groupby('Neighborhood').mean().reset_index()
print(toronto_grouped.shape)
toronto_grouped.head()

(38, 234)


,Neighborhood,Yoga Studio,Accessories Store,Adult Boutique,Afghan Restaurant,Airport,Airport Food Court,Airport Gate,Airport Lounge,Airport Service,Airport Terminal,American Restaurant,Antique Shop,Aquarium,Arepa Restaurant,Art Gallery,Art Museum,Arts & Crafts Store,Asian Restaurant,Auto Workshop,BBQ Joint,Baby Store,Bagel Shop,Bakery,Bank,Bar,Baseball Stadium,Basketball Stadium,Beach,Beer Bar,Beer Store,Belgian Restaurant,Bistro,Boat or Ferry,Bookstore,Boutique,Brazilian Restaurant,Breakfast Spot,Brewery,Bubble Tea Shop,Building,Burger Joint,Burrito Place,Bus Line,Butcher,Café,Cajun / Creole Restaurant,Caribbean Restaurant,Cheese Shop,Chinese Restaurant,Chocolate Shop,Church,Climbing Gym,Clothing Store,Cocktail Bar,Coffee Shop,College Arts Building,College Gym,College Rec Center,Comfort Food Restaurant,Comic Shop,Concert Hall,Convenience Store,Cosmetics Shop,Coworking Space,Creperie,Cuban Restaurant,Cupcake Shop,Dance Studio,Deli / Bodega,Department Store,Dessert Shop,Dim Sum Restaurant,Diner,Discount Store,Dog Run,Doner Restaurant,Donut Shop,Dumpling Restaurant,Eastern European Restaurant,Electronics Store,Ethiopian Restaurant,Event Space,Falafel Restaurant,Farmers Market,Fast Food Restaurant,Filipino Restaurant,Fish & Chips Shop,Fish Market,Flea Market,Flower Shop,Food,Food & Drink Shop,Food Court,Food Truck,Fountain,French Restaurant,Fried Chicken Joint,Fruit & Vegetable Store,Furniture / Home Store,Gaming Cafe,Garden,Garden Center,Gas Station,Gastropub,Gay Bar,General Entertainment,General Travel,German Restaurant,Gift Shop,Gluten-free Restaurant,Gourmet Shop,Greek Restaurant,Grocery Store,Gym,Gym / Fitness Center,Harbor / Marina,Hawaiian Restaurant,Health & Beauty Service,Health Food Store,Historic Site,History Museum,Hobby Shop,Home Service,Hookah Bar,Hostel,Hotel,Hotel Bar,Hotpot Restaurant,Ice Cream Shop,Indian Restaurant,Indie Movie Theater,Intersection,Irish Pub,Italian Restaurant,Japanese Restaurant,Jazz Club,Jewelry Store,Jewish Restaurant,Juice Bar,Korean Restaurant,Lake,Latin American Restaurant,Light Rail Station,Lingerie Store,Liquor Store,Lounge,Mac & Cheese Joint,Malay Restaurant,Market,Martial Arts Dojo,Mediterranean Restaurant,Men's Store,Metro Station,Mexican Restaurant,Middle Eastern Restaurant,Miscellaneous Shop,Modern European Restaurant,Molecular Gastronomy Restaurant,Monument / Landmark,Movie Theater,Museum,Music Venue,New American Restaurant,Nightclub,Noodle House,Office,Opera House,Optical Shop,Organic Grocery,Park,Performing Arts Venue,Persian Restaurant,Pet Store,Pharmacy,Piano Bar,Pizza Place,Plane,Playground,Plaza,Portuguese Restaurant,Poutine Place,Pub,Ramen Restaurant,Record Shop,Recording Studio,Rental Car Location,Restaurant,Sake Bar,Salad Place,Salon / Barbershop,Sandwich Place,Scenic Lookout,Sculpture Garden,Seafood Restaurant,Shoe Store,Shopping Mall,Skate Park,Skating Rink,Smoke Shop,Smoothie Shop,Snack Place,Soup Place,Southern / Soul Food Restaurant,Spa,Speakeasy,Sporting Goods Shop,Sports Bar,Stadium,Stationery Store,Steakhouse,Strip Club,Supermarket,Sushi Restaurant,Swim School,Taco Place,Tailor Shop,Taiwanese Restaurant,Tanning Salon,Tapas Restaurant,Tea Room,Tennis Court,Thai Restaurant,Theater,Thrift / Vintage Store,Toy / Game Store,Trail,Train Station,Vegetarian / Vegan Restaurant,Video Game Store,Vietnamese Restaurant,Wine Bar,Wings Joint,Women's Store
0,"Adelaide, King",0.0,0.01,0.0,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.04,0.0,0.0,0.0,0.010000,0.01,0.0,0.02,0.0000,0.0,0.0,0.000000,0.020000,0.0,0.030000,0.0,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.01,0.000000,0.01,0.020000,0.0000,0.0,0.01,0.020000,0.010000,0.0,0.0000,0.060000,0.0,0.000000,0.000000,0.0,0.0,0.0,0.000000,0.020000,0.010000,0.060000,0.0,0.0,0.0,0.000000,0.0000,0.020000,0.000000,0.020000,0.0,0.000000,0.0,0.0,0.0,0.02,0.01,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.01,0.0,0.0,0.000000,0.000000,0.0000,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.01,0.0,0.000000,0.000000,0.0,0.0,0.010000,0

# Define function which sorts venues in descending order.
This function will be useful later for examining clusters.

In [18]:
# First, let's write a function to sort the venues in descending order.


def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]


# Now let's create the new dataframe and display the top 10 venues for each neighborhood.

num_top_venues = int(10)

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
column_names = ['Neighborhood']
for ind in np.arange(num_top_venues):
    try:
        column_names.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        column_names.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
neighborhoods_venues_sorted = pd.DataFrame(columns=column_names)
neighborhoods_venues_sorted['Neighborhood'] = toronto_grouped['Neighborhood']

for ind in range(0,toronto_grouped.shape[0]):
    neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(toronto_grouped.iloc[ind, :], num_top_venues)

neighborhoods_venues_sorted.set_index("Neighborhood", inplace = True)

neighborhoods_venues_sorted.head()

,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
Neighborhood,,,,,,,,,,
"Adelaide, King",Café,Coffee Shop,Steakhouse,American Restaurant,Thai Restaurant,Hotel,Bar,Restaurant,Gym,Deli / Bodega
Berczy Park,Coffee Shop,Cocktail Bar,Café,Cheese Shop,Beer Bar,Steakhouse,Seafood Restaurant,Farmers Market,Restaurant,Bakery
"Brockton, Exhibition Place",Coffee Shop,Breakfast Spot,Café,Pet Store,Grocery Store,Burrito Place,Caribbean Restaurant,Climbing Gym,Performing Arts Venue,Stadium
Business reply mail Processing Centre969 Eastern,Light Rail Station,Auto Workshop,Park,Pizza Place,Recording Studio,Butcher,Restaurant,Burrito Place,Brewery,Skate Park
"CN Tower, Bathurst Quay, Island airport, Harbourfront West, King and Spadina, Railway Lands",Airport Lounge,Airport Terminal,Airport Service,Harbor / Marina,Boat or Ferry,Sculpture Garden,Plane,Boutique,Airport Gate,Airport


# Now time for clustering!
Divide neighborhood into 5 clusters using k-menas algorithm.

In [64]:
# set number of clusters
kclusters = 5


#Prepare data for clustering

toronto_grouped_clustering = toronto_grouped.drop('Neighborhood', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(toronto_grouped_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:10] 

array([3, 3, 3, 0, 3, 3, 3, 3, 3, 3], dtype=int32)

# Join table with sorted venues with cluster number.

In [24]:
toronto_merged = hoods_df.set_index("Neighborhood")

# add clustering labels

toronto_merged['Cluster Labels'] = kmeans.labels_.tolist()


# merge toronto_grouped with toronto_data to add latitude/longitude for each neighbourhood
toronto_merged = toronto_merged.join(neighborhoods_venues_sorted).reset_index()


toronto_merged.head() 

# create map
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=12)

# Print map of neighborhoods colored by clusters.

In [68]:
# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i+x+(i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(toronto_merged['Latitude'], toronto_merged['Longitude'], toronto_merged['Neighborhood'], toronto_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters



# Name the clusters. 
Develop basic statistics: count of neighborhoods in cluster, count types of venues in top 10, show percentage. Examine centers of clusters to find the most distinctive factor in the group.


# Cluster 0 - "Coffee places"
There is a coffee shop in all 4 neighborhoods in this cluster.


In [69]:
# Name the clusters


# Get top 10  venues in all neighborhoods and count entires.

cluster_0 = toronto_merged.loc[toronto_merged['Cluster Labels'] == 0, toronto_merged.columns[list(range(6, toronto_merged.shape[1]))]]

count_of_neighbourhoods = cluster_0.shape[0]

cluster_0 = cluster_0.values.tolist()

cluster_0_list = []

for item in cluster_0:
        cluster_0_list = cluster_0_list + item
        

cluster_0_df = pd.DataFrame({'venue':cluster_0_list, 'counter':1})
cluster_0_df = cluster_0_df.groupby('venue').count()
cluster_0_df['counter2']= ""
cluster_0_df['counter2'] = cluster_0_df['counter']/count_of_neighbourhoods
print("There are {} neighbourhoods in this cluster.".format(count_of_neighbourhoods))
cluster_0_df.sort_values('counter', ascending = False). head(10)



There are 4 neighbourhoods in this cluster.


,counter,counter2
venue,,
Coffee Shop,4,1.00
Diner,2,0.50
Café,2,0.50
Restaurant,2,0.50
Music Venue,1,0.25
Japanese Restaurant,1,0.25
Mediterranean Restaurant,1,0.25
Men's Store,1,0.25
Mexican Restaurant,1,0.25


In [70]:
# Save cluster centres in data frame. Slice out series for each cluster and sort it in descending order.

centers = pd.DataFrame(kmeans.cluster_centers_.tolist(), columns = toronto_grouped_clustering.columns.tolist())

center_0 = centers.iloc[0,:]

center_0.sort_values(ascending = False)[0:20]

Park                    0.120685
Sandwich Place          0.055060
Swim School             0.050000
Bus Line                0.050000
Dim Sum Restaurant      0.050000
Lake                    0.050000
Burger Joint            0.043155
Hotel                   0.043155
Food & Drink Shop       0.043155
Breakfast Spot          0.031250
Grocery Store           0.031250
Clothing Store          0.031250
Burrito Place           0.027530
Brewery                 0.027530
Fast Food Restaurant    0.027530
Pizza Place             0.027530
Spa                     0.015625
Farmers Market          0.015625
Recording Studio        0.015625
Light Rail Station      0.015625
Name: 0, dtype: float64

# Cluster 1 - "Moore Park"
There is only one neighborhood in this cluster, so we take it's name.

In [71]:
cluster_1 = toronto_merged.loc[toronto_merged['Cluster Labels'] == 1, toronto_merged.columns[list(range(7, toronto_merged.shape[1]))]]

count_of_neighbourhoods = cluster_1.shape[0]

cluster_1 = cluster_1.values.tolist()

cluster_1_list = []

for item in cluster_1:
        cluster_1_list = cluster_1_list + item
        

cluster_1_df = pd.DataFrame({'venue':cluster_1_list, 'counter':1})
cluster_1_df = cluster_1_df.groupby('venue').count()
cluster_1_df['counter2'] = ""
cluster_1_df['counter2'] = cluster_1_df['counter']/count_of_neighbourhoods
print("There are {} neighbourhoods in this cluster.".format(count_of_neighbourhoods))
print("The neighbourhood in this cluster is {}.".format(str(toronto_merged.loc[toronto_merged['Cluster Labels'] == 1,'Neighborhood'])[5:16]))




cluster_1_df.sort_values('counter', ascending = False). head(10)



There are 1 neighbourhoods in this cluster.
The neighbourhood in this cluster is  Moore Park.


,counter,counter2
venue,,
Dim Sum Restaurant,1,1.0
Eastern European Restaurant,1,1.0
Electronics Store,1,1.0
Ethiopian Restaurant,1,1.0
Event Space,1,1.0
Falafel Restaurant,1,1.0
Intersection,1,1.0
Park,1,1.0
Tennis Court,1,1.0


In [72]:
#Show center
center_1 = centers.iloc[1,:]

center_1.sort_values(ascending = False)[0:10]


Garden                         0.5
Home Service                   0.5
Women's Store                  0.0
Fast Food Restaurant           0.0
Farmers Market                 0.0
Falafel Restaurant             0.0
Event Space                    0.0
Ethiopian Restaurant           0.0
Electronics Store              0.0
Eastern European Restaurant    0.0
Name: 1, dtype: float64

# Cluster 2 - "Trail in a park"
The highest entires in center vector were Park and Trail.

In [73]:
cluster_2 = toronto_merged.loc[toronto_merged['Cluster Labels'] == 2, toronto_merged.columns[list(range(7, toronto_merged.shape[1]))]]

count_of_neighbourhoods = cluster_2.shape[0]

cluster_2 = cluster_2.values.tolist()

cluster_2_list = []

for item in cluster_2:
        cluster_2_list = cluster_2_list + item
        

cluster_2_df = pd.DataFrame({'venue':cluster_2_list, 'counter':1})
cluster_2_df = cluster_2_df.groupby('venue').count()
cluster_2_df['counter2'] =""
cluster_2_df['counter2'] = cluster_2_df['counter']/count_of_neighbourhoods
print("There are {} neighbourhoods in this cluster.".format(count_of_neighbourhoods))
cluster_2_df.sort_values('counter', ascending = False). head(10)



There are 2 neighbourhoods in this cluster.


,counter,counter2
venue,,
Bookstore,1,0.5
Burrito Place,1,0.5
Swim School,1,0.5
Sushi Restaurant,1,0.5
Smoothie Shop,1,0.5
Pizza Place,1,0.5
Park,1,0.5
Lake,1,0.5
Italian Restaurant,1,0.5


In [74]:
# Show center
center_2 = centers.iloc[2,:]

center_2.sort_values(ascending = False)[0:10]


Park                   0.375
Trail                  0.250
Jewelry Store          0.125
Sushi Restaurant       0.125
Playground             0.125
Women's Store          0.000
Dog Run                0.000
Doner Restaurant       0.000
Donut Shop             0.000
Dumpling Restaurant    0.000
Name: 2, dtype: float64

# Cluster 4 - "Typical Toronto"
There are 30 neighborhoods in this cluster, which means that they represent standard types of venues across whole town.

In [75]:
cluster_3 = toronto_merged.loc[toronto_merged['Cluster Labels'] == 3, toronto_merged.columns[list(range(7, toronto_merged.shape[1]))]]

count_of_neighbourhoods = cluster_3.shape[0]

cluster_3 = cluster_3.values.tolist()

cluster_3_list = []

for item in cluster_3:
        cluster_3_list = cluster_3_list + item
        

cluster_3_df = pd.DataFrame({'venue':cluster_3_list, 'counter':1})
cluster_3_df = cluster_3_df.groupby('venue').count()
cluster_3_df['counter2'] = ""
cluster_3_df['counter2'] = cluster_3_df['counter']/count_of_neighbourhoods
print("There are {} neighbourhoods in this cluster.".format(count_of_neighbourhoods))
cluster_3_df.sort_values('counter', ascending = False). head(10)


There are 30 neighbourhoods in this cluster.


,counter,counter2
venue,,
Café,16,0.533333
Italian Restaurant,12,0.400000
Restaurant,12,0.400000
Bakery,11,0.366667
Coffee Shop,9,0.300000
Pizza Place,8,0.266667
Bar,8,0.266667
Hotel,7,0.233333
Park,6,0.200000


In [76]:
#Show center of cluster
center_3 = centers.iloc[3,:]

center_3.sort_values(ascending = False)[0:10]

Coffee Shop           0.092159
Café                  0.061687
Italian Restaurant    0.025568
Restaurant            0.025326
Bar                   0.024728
Pub                   0.022551
Bakery                0.022300
Park                  0.020207
Pizza Place           0.016922
Sandwich Place        0.016297
Name: 3, dtype: float64

# Cluster 5 - "Parkdale"
There is only one neighborhood in this cluster, so we take it's name.

In [77]:
cluster_4 = toronto_merged.loc[toronto_merged['Cluster Labels'] == 4, toronto_merged.columns[list(range(7, toronto_merged.shape[1]))]]

count_of_neighbourhoods = cluster_4.shape[0]

cluster_4 = cluster_4.values.tolist()

cluster_4_list = []

for item in cluster_4:
        cluster_4_list = cluster_4_list + item
        

cluster_4_df = pd.DataFrame({'venue':cluster_4_list, 'counter':1})
cluster_4_df = cluster_4_df.groupby('venue').count()
cluster_4_df['counter2'] = ""
cluster_4_df['counter2'] =cluster_4_df['counter']/count_of_neighbourhoods
print("There are {} neighbourhoods in this cluster.".format(count_of_neighbourhoods))
cluster_4_df.sort_values('counter', ascending = False).head(10)

print("The neighbourhood in this cluster is {}.".format(str(toronto_merged.loc[toronto_merged['Cluster Labels'] == 4,'Neighborhood'])[5:14]))



There are 1 neighbourhoods in this cluster.
The neighbourhood in this cluster is  Parkdale.


In [78]:
#Show center
center_4 = centers.iloc[4,:]
center_4.sort_values(ascending = False)[0:10]

Restaurant                     0.25
Park                           0.25
Tennis Court                   0.25
Intersection                   0.25
Dim Sum Restaurant             0.00
Falafel Restaurant             0.00
Event Space                    0.00
Ethiopian Restaurant           0.00
Electronics Store              0.00
Eastern European Restaurant    0.00
Name: 4, dtype: float64

# Done!

This wa the last notebook on Segmenting and Clustering Neighborhoods in Toronto. Your feedback is very welcome!

# Thank you for reading!
